In [1]:
notebook_id = "125"

<a id="ID_top"></a>
## Merge scope data with distance calculations and filter away any empty datapoints

Workflow takes in inputs 120 and 121 (all) and merges them together. After that any distance that were not calculated are used to filter out those entries / countries.

This workflows returns dataset 125 which is ready for country level network analysis.

#### Notebook sections:
    
|| [0| Default imports](#ID_top) || [1|Part1 Reference table generation](#ID_part1) || [2|Part2 Table maintanence](#ID_part2) || [3|Part3 Table export](#ID_part3) || 

#### Import all packages that could be required

In [2]:
# %load s_package_import.py
# package library, use to ensure consistency across notebooks, refresh periodically
# general packages
import os # use with os.listdir(_path_)
import requests
import csv
import time
from datetime import datetime
from shutil import copyfile

#temp check
#from shutil import make_archive
import zipfile #notebook

# data analysis packages
import pandas as pd
pd.options.display.max_columns = None # don't truncate columns
#pd.options.display.max_rows = None

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes
#import pycountry

# custom scripts
import s_file_export
import s_filepaths
import s_un_comtrade_extract as s_un
import s_adj_matrix_plot

#=== network analysis
import networkx as nx
#=== gavity modelling
#import gme as gme

#=== distance datasets
#import wbdata


#### Import module and declare path variables
`import s_filepaths.py`

In [3]:
# import ref file
import s_filepaths

# declare local variables to work with
path_raw = s_filepaths.path_raw
path_raw_dl = s_filepaths.path_raw_dl
path_store = s_filepaths.path_store
path_live = s_filepaths.path_live

<a id="ID_part1"></a>
### Part 1 | Load and Merge
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

In [4]:
# View all files in live directory
sorted(list(os.listdir(f"{path_live}")))

['.DS_Store',
 '112_input_gme_data.csv.gzip',
 '113_input_bri_members.csv.gzip',
 '120_input_scope_data.csv.gzip',
 '121_input_de_matrix.csv.gzip',
 '121_input_de_min_max.csv.gzip',
 '121_input_di_matrix.csv.gzip',
 '121_input_di_min_max.csv.gzip',
 '121_input_lpi_matrix.csv.gzip',
 '122_input_un_com_2016_mini.csv.gzip',
 '125_input_scope_country.csv.gzip',
 '2_raw_explainer_doc.md']

**Import 120 | GME data**

In [5]:
# import gme data (120)
df_scope_raw = pd.read_csv(f"{path_live}120_input_scope_data.csv.gzip",compression="gzip")
df_scope_raw.head()

,year,country_d,iso3_d,dynamic_code_d,landlocked_d,island_d,region_d,gdp_pwt_const_d,pop_d,lat_d,lng_d,country_o,iso3_o,dynamic_code_o,landlocked_o,island_o,region_o,pop_o,lat_o,lng_o,contiguity,distance,bri_year_d,bri_flag_d
0,2016,United Arab Emirates,ARE,ARE,0,0,middle_east,NaN,NaN,25.094378,55.454674,Portugal,PRT,PRT,0,0,europe,NaN,39.310741,-10.883486,0,6154.4214,2017.0,1.0
1,2016,Azerbaijan,AZE,AZE,0,0,europe,NaN,NaN,40.362438,47.255833,Kuwait,KWT,KWT,0,0,middle_east,NaN,29.279369,47.930035,0,1236.6086,2015.0,1.0
2,2016,Belgium,BEL,BEL,0,0,europe,NaN,NaN,50.691814,4.581812,Cambodia,KHM,KHM,0,0,south_east_asia,NaN,12.320902,104.874400,0,9693.0625,NaN,NaN
3,2016,China,CHN,CHN,0,0,east_asia,NaN,NaN,35.389668,114.002470,Belarus,BLR,BLR,1,0,europe,NaN,53.604687,27.802185,0,6904.3364,2013.0,1.0
4,2016,Czech Republic,CZE,CZE,1,0,europe,NaN,NaN,49.817062,15.696862,Holy See,VAT,VAT,1,0,europe,NaN,41.900013,12.447808,0,925.0072,2015.0,1.0


**Import 121 | Distance data**

In [6]:
# import distance measures (120)
df_de = pd.read_csv(f"{path_live}121_input_de_matrix.csv.gzip",compression="gzip",index_col="index")
#df_de_minmax = pd.read_csv(f"{path_live}121_input_de_min_max.csv.gzip",compression="gzip",index_col="index")

df_di = pd.read_csv(f"{path_live}121_input_di_matrix.csv.gzip",compression="gzip",index_col="index")
#df_di_minmax = pd.read_csv(f"{path_live}121_input_di_min_max.csv.gzip",compression="gzip",index_col="index")

df_lpi = pd.read_csv(f"{path_live}121_input_lpi_matrix.csv.gzip",compression="gzip",index_col="index")
#df_di.head()

In [7]:
# add arrays as pair columns

# list of arrays
distance_array_list = [df_de,df_di,df_lpi]
# list of column names
distante_array_columns = ["economic_distance","institutional_distance","logistics"]

In [8]:
def f_array_to_column(p_df,p_df_array,p_col_name):
    # create series based on combo of iso3_o and iso3_d
    paired_values = []
    
    f_df = p_df.copy()

    for entry in list(p_df.index):
        # pair of countries in gravity dataset
        df_pair = (f_df.iloc[entry].iso3_o, f_df.iloc[entry].iso3_d)

        try: 
            distance_value = p_df_array.loc[df_pair[0],df_pair[1]]
            paired_values.append(distance_value)
        except:
            paired_values.append(np.NaN)

    f_df[p_col_name] = paired_values
    
    return f_df

In [9]:
df_scope_distance = df_scope_raw.copy()

for index,entry in enumerate(distance_array_list):
    df_scope_distance = f_array_to_column(df_scope_distance,entry,distante_array_columns[index]).copy()
    
df_scope_distance.head()

,year,country_d,iso3_d,dynamic_code_d,landlocked_d,island_d,region_d,gdp_pwt_const_d,pop_d,lat_d,lng_d,country_o,iso3_o,dynamic_code_o,landlocked_o,island_o,region_o,pop_o,lat_o,lng_o,contiguity,distance,bri_year_d,bri_flag_d,economic_distance,institutional_distance,logistics
0,2016,United Arab Emirates,ARE,ARE,0,0,middle_east,NaN,NaN,25.094378,55.454674,Portugal,PRT,PRT,0,0,europe,NaN,39.310741,-10.883486,0,6154.4214,2017.0,1.0,32364.307560,0.403792,1.331108
1,2016,Azerbaijan,AZE,AZE,0,0,europe,NaN,NaN,40.362438,47.255833,Kuwait,KWT,KWT,0,0,middle_east,NaN,29.279369,47.930035,0,1236.6086,2015.0,1.0,NaN,NaN,NaN
2,2016,Belgium,BEL,BEL,0,0,europe,NaN,NaN,50.691814,4.581812,Cambodia,KHM,KHM,0,0,south_east_asia,NaN,12.320902,104.874400,0,9693.0625,NaN,NaN,44933.919861,4.775302,1.386359
3,2016,China,CHN,CHN,0,0,east_asia,NaN,NaN,35.389668,114.002470,Belarus,BLR,BLR,1,0,europe,NaN,53.604687,27.802185,0,6904.3364,2013.0,1.0,4153.131454,0.004938,1.492458
4,2016,Czech Republic,CZE,CZE,1,0,europe,NaN,NaN,49.817062,15.696862,Holy See,VAT,VAT,1,0,europe,NaN,41.900013,12.447808,0,925.0072,2015.0,1.0,NaN,NaN,NaN


**Import 121 | calculated distance logistics column**

In [10]:
df_scope_distance["logistics_geo_distance"] = df_scope_distance["distance"] * df_scope_distance["logistics"]

**Import 122 | Flow data**

In [11]:
# Load mini comtrade dataset (trimmed down in workflow 122)
df_un_com = pd.read_csv(f"{path_live}122_input_un_com_2016_mini.csv.gzip",compression="gzip")

df_un_com_export = df_un_com[df_un_com.rgDesc == "Export"].copy()
df_un_com_export.rename(columns={"TradeValue": "Export_value"},inplace =True)

df_un_com_import = df_un_com[df_un_com.rgDesc == "Import"].copy()
df_un_com_import.rename(columns={"TradeValue": "Import_value"},inplace = True)
df_un_com_export.head()

,rtCode,rt3ISO,ptCode,pt3ISO,rgDesc,yr,Export_value
1,784,ARE,0,WLD,Export,2016,295046691148
4,784,ARE,4,AFG,Export,2016,1596610735
7,784,ARE,8,ALB,Export,2016,11867263
10,784,ARE,12,DZA,Export,2016,374836964
13,784,ARE,20,AND,Export,2016,89855


In [12]:
# merge imports
df_scope_raw_im = df_scope_distance.merge(
    df_un_com_import
    ,how = "left",left_on = ["year","iso3_d","iso3_o"],
    # the arrangement of pt and rt columns matter in the key as they indicate the flow of goods 
    # depending on whether it is imports or exports
    right_on = ["yr","rt3ISO","pt3ISO"], # for imports, the reporter is the destination, and partner is the origin
                  ).copy()

# drop uneccesary columns
df_scope_raw_im.drop(labels = ["yr","rgDesc","pt3ISO","rt3ISO","ptCode","rtCode"], axis = 1, inplace = True)

# merge exports
df_scope_raw_im_ex = df_scope_raw_im.merge(
    df_un_com_export
    ,how = "left",left_on = ["year","iso3_d","iso3_o"],
    # the arrangement of pt and rt columns matter in the key as they indicate the flow of goods 
    # depending on whether it is imports or exports
    right_on = ["yr","pt3ISO","rt3ISO"], # for imports, the reporter is the destination, and partner is the origin
                  ).copy()

# drop uneccesary columns
df_scope_raw_im_ex.drop(labels = ["yr","rgDesc"], axis = 1, inplace = True)

# drop uneccesary columns
#df_scope_raw_im_ex.drop(labels = ["yr_y","yr_x","rgDesc_y","pt3ISO_y","rt3ISO_y",""], axis = 1, inplace = True)

In [13]:
df_scope_raw_im_ex.head()

,year,country_d,iso3_d,dynamic_code_d,landlocked_d,island_d,region_d,gdp_pwt_const_d,pop_d,lat_d,lng_d,country_o,iso3_o,dynamic_code_o,landlocked_o,island_o,region_o,pop_o,lat_o,lng_o,contiguity,distance,bri_year_d,bri_flag_d,economic_distance,institutional_distance,logistics,logistics_geo_distance,Import_value,rtCode,rt3ISO,ptCode,pt3ISO,Export_value
0,2016,United Arab Emirates,ARE,ARE,0,0,middle_east,NaN,NaN,25.094378,55.454674,Portugal,PRT,PRT,0,0,europe,NaN,39.310741,-10.883486,0,6154.4214,2017.0,1.0,32364.307560,0.403792,1.331108,8192.201100,218350994.0,620.0,PRT,784.0,ARE,179403385.0
1,2016,Azerbaijan,AZE,AZE,0,0,europe,NaN,NaN,40.362438,47.255833,Kuwait,KWT,KWT,0,0,middle_east,NaN,29.279369,47.930035,0,1236.6086,2015.0,1.0,NaN,NaN,NaN,NaN,3530921.0,414.0,KWT,31.0,AZE,1235241.0
2,2016,Belgium,BEL,BEL,0,0,europe,NaN,NaN,50.691814,4.581812,Cambodia,KHM,KHM,0,0,south_east_asia,NaN,12.320902,104.874400,0,9693.0625,NaN,NaN,44933.919861,4.775302,1.386359,13438.064434,367037492.0,116.0,KHM,56.0,BEL,396804738.0
3,2016,China,CHN,CHN,0,0,east_asia,NaN,NaN,35.389668,114.002470,Belarus,BLR,BLR,1,0,europe,NaN,53.604687,27.802185,0,6904.3364,2013.0,1.0,4153.131454,0.004938,1.492458,10304.432095,435188908.0,112.0,BLR,156.0,CHN,400511800.0
4,2016,Czech Republic,CZE,CZE,1,0,europe,NaN,NaN,49.817062,15.696862,Holy See,VAT,VAT,1,0,europe,NaN,41.900013,12.447808,0,925.0072,2015.0,1.0,NaN,NaN,NaN,NaN,6045.0,NaN,NaN,NaN,NaN,NaN


<a id="ID_part2"></a>
### Part 2 | Final scope filter
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

In [14]:
# remove any entries with no distance measurement
df_scope = df_scope_raw_im_ex[~df_scope_raw_im_ex.economic_distance.isnull()].copy()

In [15]:
print(f"Country count | {len(df_scope.iso3_d.unique())}")
df_scope.iso3_d.unique()

Country count | 72


array(['ARE', 'BEL', 'CHN', 'ESP', 'GEO', 'HUN', 'IRQ', 'KGZ', 'LTU',
       'OMN', 'SRB', 'SWE', 'THA', 'UKR', 'VNM', 'AFG', 'ALB', 'ARM',
       'AUT', 'BGD', 'BGR', 'BIH', 'BLR', 'BRN', 'BTN', 'CHE', 'CZE',
       'DEU', 'DNK', 'EST', 'FIN', 'FRA', 'GRC', 'HKG', 'HRV', 'IDN',
       'IND', 'IRL', 'IRN', 'ISR', 'ITA', 'JOR', 'KAZ', 'KHM', 'KOR',
       'KWT', 'LAO', 'LBN', 'LUX', 'LVA', 'MDA', 'MKD', 'MMR', 'MNG',
       'MNE', 'MYS', 'NLD', 'NOR', 'NPL', 'PAK', 'POL', 'PRT', 'QAT',
       'ROU', 'RUS', 'SAU', 'SGP', 'SVK', 'TJK', 'TKM', 'TUR', 'UZB'],
      dtype=object)

In [16]:
# remove any islands or areas we know will not connect(i.e. Korea)
ireland_index = df_scope[(df_scope.country_d == "Ireland") | (df_scope.country_o == "Ireland")].index
# drop Ireland index
df_scope.drop(ireland_index, inplace= True)

In [17]:
# remove any islands or areas we know will not connect(i.e. Korea)
korea_index = df_scope[(df_scope.iso3_d == "KOR") | (df_scope.iso3_o == "KOR")].index
# drop Ireland index
df_scope.drop(korea_index, inplace= True)

In [18]:
# amend russia-poland contiguity
rus_pol_index = df_scope[((df_scope.iso3_d == "RUS") & (df_scope.iso3_o == "POL")) |
         ((df_scope.iso3_o == "RUS") & (df_scope.iso3_d == "POL"))      
        ].index

df_scope.loc[rus_pol_index,"contiguity"] = 0

In [19]:
# amend russia-lithuania contiguity
rus_pol_index = df_scope[((df_scope.iso3_d == "RUS") & (df_scope.iso3_o == "LTU")) |
         ((df_scope.iso3_o == "RUS") & (df_scope.iso3_d == "LTU"))      
        ].index

df_scope.loc[rus_pol_index,"contiguity"] = 0

df_scope[((df_scope.iso3_d == "RUS") & (df_scope.iso3_o == "LTU")) |
         ((df_scope.iso3_o == "RUS") & (df_scope.iso3_d == "LTU"))      
        ]

,year,country_d,iso3_d,dynamic_code_d,landlocked_d,island_d,region_d,gdp_pwt_const_d,pop_d,lat_d,lng_d,country_o,iso3_o,dynamic_code_o,landlocked_o,island_o,region_o,pop_o,lat_o,lng_o,contiguity,distance,bri_year_d,bri_flag_d,economic_distance,institutional_distance,logistics,logistics_geo_distance,Import_value,rtCode,rt3ISO,ptCode,pt3ISO,Export_value
4240,2016,Lithuania,LTU,LTU,0,0,europe,NaN,NaN,55.406567,23.602331,Russia,RUS,RUS,0,0,eurasia,NaN,56.643204,73.978935,0,2021.7047,2017.0,1.0,6799.53208,2.520026,1.474681,2981.369509,3.793243e+09,643.0,RUS,440.0,LTU,2.554986e+09
6096,2016,Russia,RUS,RUS,0,0,eurasia,NaN,NaN,56.643204,73.978935,Lithuania,LTU,LTU,0,0,europe,NaN,55.406567,23.602331,0,2021.7047,NaN,1.0,6799.53208,2.520026,1.474681,2981.369509,4.674725e+08,440.0,LTU,643.0,RUS,3.371119e+09


Optional removal of Russian and Chinese contiguity.<br>
Not pursued as this is a large border in the North-East

In [20]:
# amend russia-china contiguity
"""
rus_pol_index = df_scope[((df_scope.iso3_d == "RUS") & (df_scope.iso3_o == "CHN")) |
         ((df_scope.iso3_o == "RUS") & (df_scope.iso3_d == "CHN"))      
        ].index

df_scope.loc[rus_pol_index,"contiguity"] = 0

df_scope[((df_scope.iso3_d == "RUS") & (df_scope.iso3_o == "CHN")) |
         ((df_scope.iso3_o == "RUS") & (df_scope.iso3_d == "CHN"))      
        ]
"""
print("RUS-CHN not pursued")

RUS-CHN not pursued


In [21]:
print(f"Country count | {len(df_scope.iso3_d.unique())}")
df_scope.head()

Country count | 70


,year,country_d,iso3_d,dynamic_code_d,landlocked_d,island_d,region_d,gdp_pwt_const_d,pop_d,lat_d,lng_d,country_o,iso3_o,dynamic_code_o,landlocked_o,island_o,region_o,pop_o,lat_o,lng_o,contiguity,distance,bri_year_d,bri_flag_d,economic_distance,institutional_distance,logistics,logistics_geo_distance,Import_value,rtCode,rt3ISO,ptCode,pt3ISO,Export_value
0,2016,United Arab Emirates,ARE,ARE,0,0,middle_east,NaN,NaN,25.094378,55.454674,Portugal,PRT,PRT,0,0,europe,NaN,39.310741,-10.883486,0,6154.4214,2017.0,1.0,32364.307560,0.403792,1.331108,8192.201100,2.183510e+08,620.0,PRT,784.0,ARE,1.794034e+08
2,2016,Belgium,BEL,BEL,0,0,europe,NaN,NaN,50.691814,4.581812,Cambodia,KHM,KHM,0,0,south_east_asia,NaN,12.320902,104.874400,0,9693.0625,NaN,NaN,44933.919861,4.775302,1.386359,13438.064434,3.670375e+08,116.0,KHM,56.0,BEL,3.968047e+08
3,2016,China,CHN,CHN,0,0,east_asia,NaN,NaN,35.389668,114.002470,Belarus,BLR,BLR,1,0,europe,NaN,53.604687,27.802185,0,6904.3364,2013.0,1.0,4153.131454,0.004938,1.492458,10304.432095,4.351889e+08,112.0,BLR,156.0,CHN,4.005118e+08
5,2016,Spain,ESP,ESP,0,0,europe,NaN,NaN,39.231297,-4.263840,Belgium,BEL,BEL,0,0,europe,NaN,50.691814,4.581812,0,1374.9351,NaN,NaN,11326.515268,1.841935,1.270506,1746.863638,7.757618e+09,56.0,BEL,724.0,ESP,1.049398e+10
6,2016,Georgia,GEO,GEO,0,0,europe,NaN,NaN,42.079079,42.981064,Iran,IRN,IRN,0,0,middle_east,NaN,33.925423,52.192562,0,1097.2799,2016.0,1.0,1153.061729,2.290772,1.630717,1789.352575,8.445334e+07,364.0,IRN,268.0,GEO,7.591084e+07


In [22]:
df_scope.iso3_d.unique()

array(['ARE', 'BEL', 'CHN', 'ESP', 'GEO', 'HUN', 'IRQ', 'KGZ', 'LTU',
       'OMN', 'SRB', 'SWE', 'UKR', 'VNM', 'AFG', 'ALB', 'ARM', 'AUT',
       'BGD', 'BGR', 'BIH', 'BLR', 'BRN', 'BTN', 'CHE', 'CZE', 'DEU',
       'DNK', 'EST', 'FIN', 'FRA', 'GRC', 'HKG', 'HRV', 'IDN', 'IND',
       'IRN', 'ISR', 'ITA', 'JOR', 'KAZ', 'KHM', 'KWT', 'LAO', 'LBN',
       'LUX', 'LVA', 'MDA', 'MKD', 'MMR', 'MNG', 'MNE', 'MYS', 'NLD',
       'NOR', 'NPL', 'PAK', 'POL', 'PRT', 'QAT', 'ROU', 'RUS', 'SAU',
       'SGP', 'SVK', 'THA', 'TJK', 'TKM', 'TUR', 'UZB'], dtype=object)

***

<a id="ID_part3"></a>
### Part 3 | Export
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

In [23]:
# Export for visualisation
file_name = f"scope_country"
s_file_export.f_df_export(df_scope,file_name,p_file_id=f"{notebook_id}_")

Export | ../Data/1_raw_processed_backup/125_store_scope_country_20200822_1707.csv | COMPLETE
COPY   | ../Data/2_raw_processed_input/125_input_scope_country.csv.gzip | COMPLETE
